# Statistical analysis
Questions => are fus1 different from 0 in early timepoints: t-test

=> trend of all proteins : linear regression +t-test 
 
 => are exocyst proteins getting farther away from Myo52 centroid signal : linear regression + t-test
 
 => do two proteins have the same trajectories? permutation test
          
To discuss with Frederic => consider each point individually : clustering => are all points in same group found in same cluster? 
PCA on individual POI/ all => composante variability ? 

NOTE: either prepare .csv file here and export towards R for analysis or perform analysis directly in python (linear regression + t-test) 

In [ ]:
#general infos considering experiement 
#files containing fusion events folder
path= '/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/' 
files = ['Prm1','Dni1','Dni2','Fus1','Exo70', 'Myo51', 'Cdc8','Exo84']
Myo= ['Myo52_M', 'Myo52_P']
RefM= '_B'
RefP= '_R'

In [4]:
# import all necessary packages 
from trajalign.traj import Traj
from trajalign.average import load_directory
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import norm as norm


## datasets preparation (timepoints vs d_PPOI distances)

load directed_distances traj files
Linear regression => consider all points of the array 
 => 2 linear regression on fus1? 
 Mean comparison => bin data in 2 groups : early fusion (t=[,] )/ late fusion (t= [,0[

column: timepoints + n° Replicats 
timepoints => np.linspace(-60,0,21)


In [3]:
#load directed distances trajectories files 

def replicats_nb(POI):
    rep_list = [ folder for folder in os.listdir(path+POI+'/aligned_trajectories/') if 'Rep_' in folder ]
    r= len(rep_list)
    return r

def get_data(POI,rep) :
    directory_rep = os.listdir( path+POI+'/aligned_trajectories/Rep_'+str(rep+1)+'/') #absolute path of directory to charge data from
    f = [ folder for folder in directory_rep if 'cell_' in folder ] # 'cell_' -> folder nam to list
    return f 

n= len(files) 
Rep= []
for i in range(n):
    r=replicats_nb(files[i])
    Rep.append(r)


folders=[[] for _ in range(n)] #lists for POI rep folders => 1
Cells=[[] for _ in range(n)] #lists of trajectories for each events for each POI of each replicate

for i in range(len(Rep)):
    folders[i]=[[] for _ in range(Rep[i])] 
    Cells[i]=[[] for _ in range(Rep[i])] 

# list all cell_ folders for each POI replicats

for i in range(n) :
    for r in range(Rep[i]):
        f=get_data(files[i],r)
        folders[i][r]= f 
        
ct=0
for f in folders:
    r=1
    i=0 
    for rep in f:
        for event in rep: 
            t_l = load_directory(path+files[ct] +'/aligned_trajectories/Rep_'+str(r)+'/'+ event ,pattern= 'dir_dist', comment_char = '#', t_unit='min' , frames = 0 ,t= 1, coord = ( 2 , 3 ) , f = 4 ,coord_err=(5 ,6),f_err = 7 ,coord_unit = 'pxl' )
            Cells[ct][i].extend(t_l)
        r+=1
        i+=1
    ct+=1
    

array([-60., -57., -54., -51., -48., -45., -42., -39., -36., -33., -30.,
       -27., -24., -21., -18., -15., -12.,  -9.,  -6.,  -3.,   0.])

In [ ]:
# order trajectories per list according to measured distance
vect_d_P_POI= [[] for _ in range(n)] # directed distance from Myo52_P towards POI for each events
vect_d_P_M= [[] for _ in range(n)]   # directed distance from Myo52_P towards Myo52_M for each events
vect_d_M_POI= [[] for _ in range(n)]  #  directed distance from Myo52_M towards POI for each events 
for i in range(len(Rep)):
    vect_d_P_POI[i]=[[] for _ in range(Rep[i])] 
    vect_d_M_POI[i]=[[] for _ in range(Rep[i])] 
    vect_d_P_M[i]= [[] for _ in range(Rep[i])]
    
for i in range(n):
        for r in range (len(Cells[i])):
            for t in Cells[i][r]:
                if "P_POI" in t.annotations() ['file'] : 
                    vect_d_P_POI[i][r].append(t)
                elif "M_POI" in t.annotations() ['file'] :
                    vect_d_M_POI[i][r].append(t)
                else : 
                    vect_d_P_M[i][r].append(t)
                    

Compute csv file, notes:

=> needs 21 timepoints, POI name, REP, measured distance
=> aim: save df with all POI infos (All replicats) independant for all POI and with dd_PM 
        function (list, file,folder, Rep) [probably] => needs loop to create vector timepoints

In [25]:
#create empty dataframe with right colums
colnames=['Event','Protein','Replicats','MDist'] # infos about the fusion event, event = cell folder id, Dist=directed distance measured
tp=np.linspace(-60,0,21) # to add timepoints as strings
for i in range(len(tp)):
    t=str(tp[i])
    colnames.append(t)

df=pd.DataFrame(columns=colnames) #create the empty dataframe

#df.to_csv('filename.csv')